In [ ]:
!pip install langchain
!pip install sentence-transformers
!pip install faiss-cpu
!pip install boto3
!pip install tqdm

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install langchain[bedrock]

In [ ]:
!pip install faiss-gpu

In [ ]:
import json
import time
import random
from tqdm import tqdm
from typing import Optional, List
from langchain.embeddings import HuggingFaceEmbeddings
import boto3
from botocore.exceptions import ClientError
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import Generation, LLMResult
from pydantic import Extra
from typing import Optional, List
from pydantic import Field
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import Generation, LLMResult

# Σύνδεση με AWS Bedrock Runtime
bedrock = boto3.client("bedrock-runtime", region_name="us-west-2")


class BedrockLLM(BaseLLM):
    model_id: str = Field(...)
    region: str = Field(...)

    def __init__(self, model_arn: str, region: str = "us-west-2"):
        # model_id = model_arn.split("/")[-1]  # <-- ΤΟ ΣΩΣΤΟ!
        # print(model_id)
        super().__init__(model_id=model_arn, region=region)
        object.__setattr__(self, "client", boto3.client("bedrock-runtime", region_name=region))

    @property
    def _llm_type(self) -> str:
        return "bedrock-custom"

    # Μέθοδος για αποστολή prompt στο Bedrock και λήψη απάντησης
    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        generations = []
        for prompt in prompts:
            for attempt in range(5):
                try:
                    body = {
                        "prompt": prompt,
                        "max_gen_len": 512,
                        "temperature": 0.0,
                        "top_p": 0.9
                    }
                    
                    response = self.client.invoke_model(
                        modelId=self.model_id
                        ,  # <-- ΠΡΟΣΟΧΗ! ΟΧΙ ARN
                        body=json.dumps(body).encode("utf-8"),
                        contentType="application/json",
                        accept="application/json"
                    )
                    result = json.loads(response["body"].read())
                    text = result.get("generation") or result.get("outputs", [{}])[0].get("text", str(result))
                    generations.append([Generation(text=text)])
                    break
                except self.client.exceptions.ModelNotReadyException:
                    wait = 2 ** attempt + random.uniform(0, 1)
                    print(f"Model not ready. Retrying in {wait:.1f} seconds...")
                    time.sleep(wait)
                except Exception as e:
                    raise RuntimeError(f"Client error during invoke: {str(e)}")
        return LLMResult(generations=generations)


# Φλορτωση Dataset
input_path = "sagemaker_ft500vers2 (1) (1).jsonl"
with open(input_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

# Ορισμός διανυσματικού μοντέλου
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

# Φόρτωση ανακτητή
retriever = FAISS.load_local(
    "e5largev2_rag_db",
    embedding_model,
    allow_dangerous_deserialization=True
).as_retriever()

# Επιλογή LLM
llm = BedrockLLM(
    #model_arn = "meta.llama3-1-70b-instruct-v1:0",
    model_arn="arn:aws:bedrock:us-west-2:471112783210:imported-model/y9xa363z3o7r",
    #model_arn="arn:aws:bedrock:us-west-2:471112783210:imported-model/1tfsf1vs44wd",
    region="us-west-2"
)

# Prompt templates
prompt_step1 = PromptTemplate(
    input_variables=["question"],
    template="""
Given the following clinical case and question, write a concise summary focusing only on the relevant clinical findings:

{question}

SUMMARY:
"""
)

prompt_step2 = PromptTemplate(
    input_variables=["summary"],
    template="""
You are a medical expert. Based on the clinical summary below, provide the most likely diagnosis with 2–3 reasonable differential diagnoses. Explain your reasoning step by step.

CLINICAL SUMMARY:
{summary}

DIAGNOSIS:
"""
)

prompt_step3 = PromptTemplate(
    input_variables=["summary", "diagnosis", "context"],
    template="""
Based on the following:
- Summary: {summary}
- Most likely diagnosis: {diagnosis}
- Relevant guideline info: {context}

Provide short-term and long-term management recommendations, including risk scores if applicable.
"""
)

prompt_step4 = PromptTemplate(
    input_variables=["question", "summary", "management"],
    template="""
Based on the following case and options:

CASE:
{question}

SUMMARY:
{summary}

MANAGEMENT PLAN:
{management}

Select the best answer choice and justify why it's correct and why the others are incorrect. Begin your answer with the letter (e.g., A.)

FINAL ANSWER:
"""
)

# Αξιολόγηση
output_path = "validation_answers_e5large_multistep_reasoning_ft&nocontext.jsonl"
with open(output_path, "w", encoding="utf-8") as fout:
    for example in tqdm(dataset):
        question = example["model_input"]
        # print(question)
        step1_output = LLMChain(llm=llm, prompt=prompt_step1).run(question=question)
        step2_output = LLMChain(llm=llm, prompt=prompt_step2).run(summary=step1_output)

        docs = retriever.get_relevant_documents(step1_output + step2_output)
        context = "\n".join([doc.page_content for doc in docs[:3]])

        step3_output = LLMChain(llm=llm, prompt=prompt_step4).run(
            summary=step1_output,
            diagnosis=step2_output,
            context=context
        )

        step4_output = LLMChain(llm=llm, prompt=prompt_step5).run(
            question=question,
            summary=step1_output,
            management=step4_output
        )
        # print(step5_output)

        output = {
            "question": question,
            "target_output": example.get("target_output", ""),
            "step1_summary": step1_output,
            "step2_diagnosis": step2_output,
            "step3_management": step4_output,
            "step4_final_answer": step5_output
        }
        fout.write(json.dumps(output) + "\n")


Ο σχεδιασμός του προτεινόμενου pipeline βασίζεται σε αρχές πολυπρακτορικών συστημάτων (multi-agent workflows), όπου κάθε υποσύστημα εκτελεί έναν σαφώς καθορισμένο, ανεξάρτητο ρόλο εντός της συνολικής ροής εργασίας. Η προσέγγιση αυτή ευθυγραμμίζεται με το μοντέλο «Modular Agent Design» που περιγράφει η Anthropic (2024), το οποίο προωθεί τον διαχωρισμό της λογικής σε διακριτά και αυτόνομα modules, έτσι ώστε κάθε agent να είναι όχι μόνο επαναχρησιμοποιήσιμος, αλλά και πλήρως αξιολογήσιμος ως προς την απόδοσή του. Ένα τέτοιο design διευκολύνει την ανίχνευση σφαλμάτων, την ενσωμάτωση μηχανισμών debugging και την εφαρμογή targeted βελτιώσεων χωρίς να επηρεάζεται το σύνολο του συστήματος. Επιπλέον, προωθεί τη δυνατότητα παραλληλισμού και επεκτασιμότητας, κάτι ιδιαίτερα σημαντικό σε περιβάλλοντα όπως η κλινική υποβοηθούμενη λήψη αποφάσεων.

Ένα από τα βασικά καινοτόμα χαρακτηριστικά του συστήματος είναι η στρατηγική τοποθέτηση του retrieval βήματος όχι στην αρχική ερώτηση (prompt-level RAG), αλλά μετά από ένα πρώτο κύκλο reasoning που περιλαμβάνει σύνοψη και διάγνωση. Η προσέγγιση αυτή συνδέεται με μεθοδολογίες όπως το RAG-Fusion (Press et al., 2023), που ενισχύουν την ανάκτηση πληροφοριών μέσω query reformulation. Σύμφωνα με τις αρχές αυτές, η άμεση χρήση της αρχικής ερώτησης ενδέχεται να οδηγήσει σε retrieval documents που δεν απαντούν ουσιαστικά το ζητούμενο, ενώ η ενδιάμεση επεξεργασία (με τη μορφή structured reasoning) επιτρέπει τη δημιουργία πιο στοχευμένων queries για αναζήτηση. Παρόμοια συμπεράσματα καταγράφονται και στη μελέτη των Santhanam et al. (2023), όπου η χρήση intermediate thought steps πριν το retrieval οδηγεί σε αυξημένη σχετικότητα και απόδοση του τελικού συστήματος. Με άλλα λόγια, το σύστημα εφαρμόζει μια μορφή "reasoning-first RAG", η οποία επιτρέπει πιο context-aware ανακτήσεις και βελτιώνει την ακρίβεια των τελικών απαντήσεων.

Η χρήση προκαθορισμένων prompts (prompt templates) ανά reasoning phase εντάσσεται στη στρατηγική του structured prompting, όπως παρουσιάζεται από τους Wei et al. (2022) και άλλους. Κάθε φάση (π.χ. clinical summary, diagnosis generation, guideline-based management, final decision) υποστηρίζεται από ένα tailor-made prompt που οδηγεί το LLM σε συγκεκριμένο τύπο reasoning. Αυτή η πρακτική όχι μόνο μειώνει το hallucination risk, αλλά διευκολύνει και τη traceability κάθε παραγόμενου reasoning βήματος. Επιπλέον, ενισχύεται η explainability της συνολικής απάντησης, καθώς η απόφαση τεκμηριώνεται σε βάση ενδιάμεσων reasoning stages. Τελική αξιολόγηση της απάντησης μπορεί να γίνει αυτόματα ή ημι-αυτόματα, χρησιμοποιώντας frameworks όπως το G-Eval (Liu et al., 2023) ή το RAGAS (Gao et al., 2023), τα οποία επιτρέπουν την αποτίμηση consistency, faithfulness και groundedness, ιδίως όταν υπάρχουν ενδιάμεσα structured outputs.

Αναφορικά με την ανάκτηση εγγράφων, αξιοποιείται ένα εξειδικευμένο vector store με embeddings από το GatorTron (Yang et al., 2022), ένα LLM σχεδιασμένο για τον ιατρικό τομέα. Η επιλογή αυτή ενισχύει την απόδοση του retriever καθώς το vector space είναι προσαρμοσμένο σε κλινική γλώσσα και φρασεολογία. Με τον τρόπο αυτό, μειώνεται το semantic drift που παρατηρείται συχνά όταν χρησιμοποιούνται general-purpose embeddings (όπως BERT ή MiniLM) σε εξειδικευμένα domains. Η χρήση domain-specific models έχει τεκμηριωθεί βιβλιογραφικά ως κρίσιμος παράγοντας για επιτυχημένη γνώση ανάκτησης (Gao et al., 2023; Yang et al., 2022).

Για την τελική παραγωγή της απάντησης χρησιμοποιείται το μοντέλο LLaMA3-70B-instruct μέσω της πλατφόρμας AWS Bedrock. Το μοντέλο αυτό, ως state-of-the-art open-weight LLM με fine-tuning σε instruction-following tasks, προσφέρει υψηλή απόδοση σε σύνθετα reasoning prompts, ιδίως όταν χρησιμοποιείται με step-by-step chains. Η απόφαση να υλοποιηθεί το pipeline με modular chains (LLMChain σε κάθε βήμα) αντί για μονολιθική prompting δομή επιτρέπει την ευελιξία στην τροποποίηση και επέκταση του workflow, όπως ενδεικνύουν οι Press et al. (2023) και οι οδηγίες της Anthropic (2024).

Βιβλιογραφία:

Anthropic. (2024). Building Effective Agents. https://www.anthropic.com/engineering/building-effective-agents

Press, O., Du, Y., Liu, P. J., & Levy, O. (2023). RAG-Fusion: Towards Information-Rich Answers from Retrieval-Augmented Generation. arXiv:2305.12987

Santhanam, K., Zeng, A., Zhang, Z., et al. (2023). Augmented Language Models: A Survey. arXiv:2302.07842

Wei, J., Wang, X., Schuurmans, D., et al. (2022). Chain-of-Thought Prompting Elicits Reasoning in Large Language Models. arXiv:2201.11903

Liu, S., Wu, Y., Gao, J., & Liu, J. (2023). G-Eval: NLG Evaluation using GPT-4 with Better Human Alignment. arXiv:2303.16634

Gao, J., Liu, S., Wang, X., et al. (2023). RAGAS: An Evaluation Framework for Retrieval-Augmented Generation. arXiv:2309.00393

Yang, X., et al. (2022). GatorTron: A Large Language Model for Clinical Natural Language Processing. arXiv:2204.1235


